In [ ]:
pip install sqlalchemy

In [5]:
#Importación de librerias
import json
from argparse import ArgumentParser
import requests
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from bson import json_util, ObjectId
import couchdb
import mysql.connector as msql
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [6]:
#Crea la conexión con MySQL y las variables que nos permiten realizar las consultas SQL 
connections = msql.connect(host = 'localhost', user = 'root', password = 'Adrian')
if connections.is_connected():
    cursor = connections.cursor()
    print("Conexión con la base de datos realizada")
else:
    print("Conexion Rechazada",connections)

Conexión con la base de datos realizada


In [7]:
#Realiza una consulta para verificar la existencia de bases de datos en MySql
consulta= pd.read_sql("show databases;", connections)
consulta

,Database
0,importar
1,information_schema
2,mysql
3,performance_schema
4,sakila
5,sys
6,tablaejemplo
7,terrorism
8,world


In [9]:
#Realiza la conexión apuntando a la base de datos deseada
connections = msql.connect(host = 'localhost', user = 'root', password = 'Adrian', database ='terrorism')
if connections.is_connected():
    cursor = connections.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("Se conectó a la base de datos: ", record)

Se conectó a la base de datos:  ('terrorism',)


In [10]:
#Realiza una consulta para verificar la existencia de bases de datos creada en MySql
consultaTable= pd.read_sql("SHOW TABLES;", connections)
consultaTable

,Tables_in_terrorism
0,terrorism


In [11]:
#realiza una consulta SQL para extraer la información de la tabla en MySQL
tomongo= pd.read_sql("SELECT * FROM terrorism", connections)
tomongo

,eventid,iyear,imonth,iday,country_txt,region,region_txt,provstate,city,attacktype1,...,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,weapsubtype1,weapsubtype1_txt,weapdetail
0,"1,97E+11",1970,7,2,Dominican Republic,2,Central America & Caribbean,,Santo Domingo,1,...,Private Citizens & Property,68,Named Civilian,,Julio Guzman,58,Dominican Republic,,,
1,"1,97E+11",1970,0,0,Mexico,1,North America,Federal,Mexico city,6,...,Government (Diplomatic),45,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21,Belgium,,,
2,"1,97E+11",1970,1,0,Philippines,5,Southeast Asia,Tarlac,Unknown,1,...,Journalists & Media,54,Radio Journalist/Staff/Facility,Voice of America,Employee,217,United States,,,
3,"1,97E+11",1970,1,0,Greece,8,Western Europe,Attica,Athens,3,...,Government (Diplomatic),46,Embassy/Consulate,,U.S. Embassy,217,United States,16,Unknown Explosive Type,Explosive
4,"1,97E+11",1970,1,0,Japan,4,East Asia,Fukouka,Fukouka,7,...,Government (Diplomatic),46,Embassy/Consulate,,U.S. Consulate,217,United States,,,Incendiary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160131,"2,02E+11",2016,3,15,India,6,South Asia,Kerala,Panniyannur,3,...,Unknown,0,,Unknown,Unknown,92,India,17,Other Explosive Type,Two steel bombs were used in the attack.
160132,"2,02E+11",2016,3,15,Nigeria,11,Sub-Saharan Africa,Delta,Odidi,3,...,Utilities,108,Oil,Nigerian National Petroleum Corporation (NNPC),Oil Facility,147,Nigeria,16,Unknown Explosive Type,
160133,"2,02E+11",2016,3,15,Libya,10,Middle East & North Africa,Sirte,Abu Hadi,6,...,Private Citizens & Property,68,Named Civilian,Not Applicable,Civilian: Muhammad al-Ghanay al-Qaddahafi,113,Libya,,,
160134,"2,02E+11",2016,3,15,India,6,South Asia,Chhattisgarh,Kumakoleng,0,...,,0,,,,0,,,,


In [12]:
#Crea la conexión con MongoDB 
try:
    client = MongoClient('mongodb://localhost:27017')
    client.server_info()
    print("Conexion exitosa")
    client.close
except pymongo.errors.ServerSelectionTimeoutError as errorTiempo:
    print("Conexión rechazada")
    
dbm = client['terrorismo']
coleccion = dbm['datas1']

Conexion exitosa


In [13]:
#Función que permite crear un documento basados en un dataframe, sodificación y posterior almacenamiento en MongoDB
def createDocsFromDF(tomongo, collection = None, insertToDB=False):
    docs = []
    fields = [col for col in tomongo.columns]
    for i in range(len(tomongo)):
        doc = {col:tomongo[col][i] for col in tomongo.columns if col != 'index'}
        for key, val in doc.items():
            if type(val) == np.int64:
                doc[key] = int(val)
            if type(val) == np.float64:
                doc[key] = float(val)
            if type(val) == np.bool_:
                doc[key] = bool(val)
        docs.append(doc) 
    if isinstance(docs, list): 
        coleccion.insert_many(docs)   
    else: 
        coleccion.insert_one(docs)
    print("guardado exitosamente")    
    return docs 

In [ ]:
#Llamamos a la función creada anteriormente y se le pasa como parametro o 
#palabra clave el dataframe que se obtuvo de la consulta SQL
createDocsFromDF(tomongo)